# gas flows model: gross trade between jurisdictions

In [ ]:
import pandas as pd
import numpy as np
import time

In [ ]:
# settings for iterative attribution process
max_iter = 5000
threshold_fract = 0.99

# when True, excludes Alaska and Hawaii from EIA data sets
contiguous_us_only = True

In [ ]:
# parameters file
parameters_file = 'GIM parameters file.xlsx'

# folder for EIA data sets
eia_path = '/Users/masoninman/Dropbox/GEM/LCA of natural gas use/EIA data for LCA of gas/'
inputs_path = '/Users/masoninman/Dropbox/GEM/LCA of natural gas use/US cities LCA of gas model/US gas model inputs/'

eia_data_2020_10_30_path = eia_path + 'EIA gas data released 2020-10-30 (production, consumption, trade)/'
trade_flows_path = eia_data_2020_10_30_path
gas_consump_data_path = eia_data_2020_10_30_path
dry_file = 'EIA gas dry production NG_PROD_SUM_A_EPG0_FPD_mmcf_a released 2020-10-30.xls'
    
states_file = 'US states and abbreviations.xlsx'

# Canada & Mexico data for dry gas production & gas production
# https://www.eia.gov/international/data/country/CAN/natural-gas/dry-natural-gas-production
# https://www.eia.gov/international/data/country/MEX/natural-gas/dry-natural-gas-production
canada_international_file = 'EIA international - Canada - dry gas production and gas consumption 1980-2019 (INT-Export-10-11-2020).csv'
mexico_international_file = 'EIA international - Mexico - dry gas production and gas consumption 1980-2019 (INT-Export-10-11-2020).csv'

In [ ]:
df = pd.read_excel(inputs_path + parameters_file, 'main parameters')
ser = df.set_index('parameter name')['parameter value']
parameters_main = ser

data_year = int(parameters_main.at['data_year'])

# Data sources:
* EIA US trade flows data: "International & Interstate Movements of Natural Gas by State" https://www.eia.gov/dnav/ng/ng_move_ist_a2dcu_nus_a.htm
* EIA US dry gas production data: "Natural Gas Gross Withdrawals and Production" https://www.eia.gov/dnav/ng/ng_prod_sum_a_EPG0_FPD_mmcf_a.htm
* EIA US gas consumption data: "Natural Gas Consumption by End Use" https://www.eia.gov/dnav/ng/ng_cons_sum_dcu_nus_a.htm
* EIA international dry gas production data & gas consumption data: https://www.eia.gov/international/overview/world

## create state list

In [ ]:
def read_states_df_create_list(states_file):
    states_df = pd.read_excel(inputs_path + states_file)
    
    if contiguous_us_only == True:
        states_df = states_df.loc[~states_df['state'].isin(['Alaska', 'Hawaii'])]
    else:
        pass
    
    states_dict = states_df.set_index('state')['abbrev'].to_dict()
    states_list = list(states_dict.keys())
    
    all_jurisdictions_list = states_list + ['Canada', 'Mexico', 'overseas']
    
    return(states_list, states_dict, all_jurisdictions_list)

## imported gas by state (or other jurisdiction)
* NG trade by state from EIA's "International & Interstate Movements of Natural Gas by State"
* https://www.eia.gov/dnav/ng/ng_move_ist_a2dcu_nus_a.htm
* (choose individual states from dropdown to download each file)

In [ ]:
def read_gas_trade_files_compile_and_clean(states_list, trade_flows_path):
    """
    Read files for gross gas imports by each state.
    
    Do not need to read Federal GOM file because it has zero gross imports (only exports).
    
    Data for Federal GOM production will show up as imports by neighboring states, e.g., Louisiana.
    
    From EIA page: International & Interstate Movements of Natural Gas by State
    https://www.eia.gov/dnav/ng/ng_move_ist_a2dcu_nus_a.htm
    
    """
    all_states_dfs = []

    # sheets in EIA state imports files are:
    # Data 1: Receipts (gross) [aka imports]
    # Data 2: Deliveries (gross) [aka exports]
    # Data 3: Net receipts [aka net imports]

    for state in states_list:
    #     print(f"processing {state}") # for db
        state_abbrev = states_dict[state]
        imports_file = f'NG_MOVE_IST_A2DCU_S{state_abbrev}_A.xls'

        imports_state = pd.read_excel(
            trade_flows_path + imports_file,
            sheet_name='Data 1', # gross receipts (aka gross imports)
            header=2
        )

        imports_state['year'] = imports_state['Date'].astype(str).str.split('-').str[0].astype(int)
        imports_state = imports_state.loc[
            (imports_state['year']>=2000) & (imports_state['year']<=data_year)]
        imports_state = imports_state.set_index('year')
        imports_state = imports_state.drop('Date', axis=1)

        for col in imports_state.columns.tolist():
            new_col = col

            new_col = new_col.replace('Natural Gas ', '')
            new_col = new_col.replace('Million Cubic Feet', 'MMcf')
            new_col = new_col.replace("  ", " ")
            new_col = new_col.replace('Receipts from', 'Receipts From')
            new_col = new_col.replace('Federal Offshore--Gulf of Mexico', 'Fed GOM')

            new_col = new_col.split('(MMcf)')[0]
            new_col = new_col.split('Natural Gas Imports + Intransit')[-1]
            new_col = new_col.split('Receipts From')[-1]
            new_col = new_col.strip()

            imports_state = imports_state.rename(columns={col: new_col})

        # to deal with error in Utah & Kansas due to duplicate columns
        if state == 'Utah' or state=='Kansas':
            cols = imports_state.columns.tolist()
            new_cols = [x for x in cols if x != f'{state} Interstate Movements: Net Receipts of']
            imports_state = imports_state[new_cols]

        if state == 'Utah':
            cols = imports_state.columns.tolist()
            new_cols = [x for x in cols if x != f'{state} Interstate Movements: Receipts of']
            imports_state = imports_state[new_cols]

        # remove state at beginning of string
        # (done this way to avoid removing 'Virginia' from 'West Virginia')
        for col in imports_state.columns.tolist():
            if col[0:len(state)] == state:
                new_col = col.replace(f'{state} ', '')
                imports_state = imports_state.rename(columns={col: new_col})

        # remove 'Imports + Intransit From ' portion
        for col in imports_state.columns.tolist():
            if col[0:len('Imports + Intransit From ')] == 'Imports + Intransit From ':
                new_col = col.replace('Imports + Intransit From ', '')
                imports_state = imports_state.rename(columns={col: new_col})

        # simplify subtotals
        if 'Net International & Interstate Receipts' in imports_state.columns:
            imports_state = imports_state.rename(columns={
                'Net International & Interstate Receipts': 'International & Interstate'})
        if 'Net Interstate Receipts' in imports_state.columns:
            imports_state = imports_state.rename(columns={
                'Net Interstate Receipts': 'Interstate'})
        if 'Net International Receipts' in imports_state.columns:
            imports_state = imports_state.rename(columns={
                'Net International Receipts': 'International'})

        # add column 'importing state'
        imports_state['importing state'] = state

        all_states_dfs += [imports_state]
        
        # TEST: LOOK FOR DUPLICATE COLUMNS
        for state_df in all_states_dfs:
            l = state_df.columns.tolist()
            dups = list(set([x for x in l if l.count(x) > 1]))
            if len(dups) > 0:
                print(f"for {state_df['importing state'].unique().tolist()[0]}, {dups}")
        # END OF TEST
        
    return all_states_dfs

In [ ]:
def compile_state_dfs_into_one_df(all_states_dfs):
    imports_all = pd.concat(all_states_dfs, sort=False)
    imports_all = imports_all.reset_index()
    imports_year = imports_all.loc[imports_all['year']==data_year].set_index('importing state').drop('year', axis=1)
    imports_year.columns.name = 'exporting state'
    imports_year.index.name = 'consuming state'
    
    return imports_year

In [ ]:
def calculate_imports_from_overseas_and_remove_extraneous_columns(imports_year):
    """
    Simplify to only keep only imports from Canada, Mexico, and overseas (which includes all other countries).
    
    Remove columns for various countries, and categories of imports. 
    
    Also, in 2019 data, at least, there was a column 'Indinana'.
    
    Seems to be a typo for Indiana. Column had no data; if empty, remove; if not empty, raise warning.
    """
    # calculate imports from overseas
    imports_year['overseas'] = imports_year['International Receipts'] - imports_year[['Canada', 'Mexico']].sum(axis=1, skipna=True)

    # check column 'Indinana':
    indinana_ser = imports_year['Indinana'].dropna()
    if len(indinana_ser) == 0:
        imports_year = imports_year.drop('Indinana', axis=1)
    else:
        print('Error!' + ' There was a typo in a column name Indinana, and there was data in it:')
        print(indinana_ser)
    
    # keep only columns in states_list + Fed GOM, Canada, Mexico, overseas
    keep_cols = states_list + ['Fed GOM', 'Canada', 'Mexico', 'overseas']
    imports_year = imports_year[keep_cols]
    
    return imports_year

In [ ]:
# run functions
(states_list, states_dict, all_jurisdictions_list) = read_states_df_create_list(states_file)
all_states_dfs = read_gas_trade_files_compile_and_clean(states_list, trade_flows_path)
imports_year = compile_state_dfs_into_one_df(all_states_dfs)
imports_year = calculate_imports_from_overseas_and_remove_extraneous_columns(imports_year)

# get overseas exports & attribute to production areas
* To compare against EIA data, need to have complete attribution of all US gas production (or all contiguous US, at least)
* Get total international gross exports of gas
* Subtract gross exports to Canada & Mexico
* Remainder will be gross overseas exports from each state
* Do attribution back to production areas for overseas exports, same as for consumption by US states & DC

### add Canada, Mexico, overseas
* use US states' gross exports data, and select only exports to Canada or Mexico
* for overseas, calculate exports other than to Canada and Mexico
* sheets in EIA state imports files are:
  * Data 1: Receipts (gross) [aka imports]
  * Data 2: Deliveries (gross) [aka exports]
  * Data 3: Net receipts [aka net imports]

In [ ]:
def read_exports_to_sel_country(sel_country, data_year):
    """
    Get data on exports to selected country from each US state.
    
    From EIA data on gas trade, sheet 'Data 2' (deliveries, aka gross exports).
    """
    
    imports_by_sel_country_dfs = [] # initialize
    
    for state in states_list:
    #     print(f"processing {state}")
        state_abbrev = states_dict[state]

        imports_file = f'NG_MOVE_IST_A2DCU_S{state_abbrev}_A.xls'
        exports_state = pd.read_excel(
            trade_flows_path + imports_file, 
            sheet_name='Data 2', # deliveries (aka gross exports)
            header=2
        )

        exports_state['year'] = exports_state['Date'].astype(str).str.split('-').str[0].astype(int)
        exports_state = exports_state.loc[
            (exports_state['year']>=2000) & (exports_state['year']<=data_year)]
        exports_state = exports_state.set_index('year')
        exports_state = exports_state.drop('Date', axis=1)

        for col in exports_state.columns.tolist():
            new_col = col
            
            # to deal with anomaly & misspelling
            new_col = new_col.replace('Lousiana Natural Gas Exports + Intransit to Portugal', 'Louisiana International Deliveries to')

            new_col = new_col.replace('Natural Gas ', '')
            new_col = new_col.replace('Million Cubic Feet', 'MMcf')
            new_col = new_col.replace("  ", " ")
            new_col = new_col.replace('Federal Offshore--Gulf of Mexico', 'Fed GOM')

            new_col = new_col.split('(MMcf)')[0]
            new_col = new_col.split('Natural Gas Imports + Intransit')[-1]
            new_col = new_col.split('Deliveries to')[-1]
            new_col = new_col.strip()
            
            new_col = new_col.replace('International Deliveries to All Countries', 'All Countries')
            new_col = new_col.replace('International Deliveries to ', '')
            
            new_col = new_col.replace('Interstate Deliveries to', '')
        
            exports_state = exports_state.rename(columns={col: new_col})
            
        dup_col = 'Exports + Intransit of'
        if dup_col in exports_state.columns:
            exports_state = exports_state.drop(f'{state} {dup_col}', axis=1)

        # remove state at beginning of string
        # (done this way to avoid removing 'Virginia' from 'West Virginia')
        for col in exports_state.columns.tolist():
            if col[0:len(state)] == state:
                new_col = col.replace(f'{state} ', '')
                exports_state = exports_state.rename(columns={col: new_col})
                
        # simplify subtotals
        if f'International Deliveries to {sel_country}' in exports_state.columns:
            exports_state = exports_state.rename(columns={
                f'International Deliveries to {sel_country}': sel_country
            })
            
        exports_state = exports_state.rename(columns={'International Deliveries': 'All Countries'})

        # keep only deliveries to selected country:
        try:
            exports_to_sel_country = exports_state.copy()[[sel_country]]
            
            # add column 'exporting juris'
            exports_to_sel_country['exporting juris'] = state
            imports_by_sel_country_dfs += [exports_to_sel_country]
        
        except:
            # no deliveries to sel_country from this state
            # print(f"for {state}, no deliveries to {sel_country}?")
            pass       
        
    # after looping through all states
    imports = pd.concat(imports_by_sel_country_dfs, sort=False)
    imports = imports.reset_index()
    
    imports_year = imports.loc[imports['year']==data_year].set_index('exporting juris').drop('year', axis=1)
    imports_year = imports_year.loc[imports_year[sel_country] > 5] # exclude negligible quantities
    
    imports_year.columns.name = 'importing juris'
        
    return imports_year

In [ ]:
def calculate_overseas_exports_from_us(
    imports_all_countries_year, 
    imports_canada_year, 
    imports_mexico_year):
    """
    Like for Canada and Mexico, calculate all US exports that go overseas.
    
    This is to be able to calculate all attributions back to US producing states, 
    and compare against EIA data for production.
    """
    
    df = pd.concat([
        imports_all_countries_year['All Countries'],
        -1*imports_canada_year['Canada'],
        -1*imports_mexico_year['Mexico']
    ], axis=1, sort=False)

    df['overseas'] = df.sum(axis=1)
    imports_overseas_year = df[['overseas']]
    
    return imports_overseas_year

### run functions

In [ ]:
# run function for Canada & Mexico
imports_canada_year = read_exports_to_sel_country('Canada', data_year)
imports_mexico_year = read_exports_to_sel_country('Mexico', data_year)

# run function for all international deliveries
# columns are, e.g., 'Louisiana Natural Gas International Deliveries to All Countries (Million Cubic Feet)'
imports_all_countries_year = read_exports_to_sel_country('All Countries', data_year)

# calculate overseas 
imports_overseas_year = calculate_overseas_exports_from_us(
    imports_all_countries_year, 
    imports_canada_year, 
    imports_mexico_year)

# add Canada, Mexico, and overseas imports to df imports_year
# they will then be in the df as "consuming states"
imports_year = imports_year.append(imports_canada_year.T, sort=False)
imports_year = imports_year.append(imports_mexico_year.T, sort=False)
imports_year = imports_year.append(imports_overseas_year.T, sort=False)

In [ ]:
# # export
# imports_year.to_csv(
#     inputs_path + 
#     f'GIM output - EIA gross imports by consuming state and immediate importer for {data_year}.csv', 
#     index=True)

## gas production (dry)

In [ ]:
def read_dry_gas_production_data(dry_file):
    # data for production for each state
    # use 'dry production'
    # units are MMcf

    dry_1 = pd.read_excel(
        eia_data_2020_10_30_path + dry_file, 
        header=2,
        sheet_name='Data 1'
    )

    dry_2 = pd.read_excel(
        eia_data_2020_10_30_path + dry_file, 
        header=2,
        sheet_name='Data 2'
    )

    dry = pd.merge(dry_1, dry_2, left_on='Date', right_on='Date')

    dry['year'] = dry['Date'].astype(str).str.split('-').str[0].astype(int)

    # only keep years from 2000 onward
    # prior to 1997 used different accounting, and there was a discrepancy in 1999
    # drop latest year (2019) because it doesn't include breakdown by state for 'Other States' (sheet 'Data 2')
    dry = dry.loc[(dry['year']>=2000) & (dry['year']<=data_year)]

    dry = dry.set_index('year')
    dry = dry.drop('Date', axis=1)
    
    return dry

In [ ]:
def clean_dry_prod_data_select_data_year(dry):
    for col in dry.columns:
        if ' Dry Natural Gas Production (MMcf)' in col:
            new_col = col.replace(' Dry Natural Gas Production (MMcf)', '')
            dry = dry.rename(columns={col: new_col})
        elif ' Dry Natural Gas Production (Million Cubic Feet)' in col:
            new_col = col.replace(' Dry Natural Gas Production (Million Cubic Feet)', '')
            dry = dry.rename(columns={col: new_col})
        elif ' Dry Natural Gas Production  (Million Cubic Feet)' in col:
            new_col = col.replace(' Dry Natural Gas Production  (Million Cubic Feet)', '')
            dry = dry.rename(columns={col: new_col})
        elif ' Natural Gas Dry Production (Million Cubic Feet)' in col:
            new_col = col.replace(' Natural Gas Dry Production (Million Cubic Feet)', '')
            dry = dry.rename(columns={col: new_col})
        elif ' Dry Production of Natural Gas (Million Cubic Feet)' in col:
            new_col = col.replace(' Dry Production of Natural Gas (Million Cubic Feet)', '')
            dry = dry.rename(columns={col: new_col})
        else:
            print(col)
            
    # drop extraneous columns for California
    # unlike others, Federal Offshore California included in California total
    for col in [
        'Calif--Onshore',
        'California--State Offshore', 
        # 'Federal Offshore California',
    ]:
        try:
            dry = dry.drop(col, axis=1)
        except:
            print(f"col wasn't in df: {col}")

    # drop extraneous columns for offshore
    for state in ['Alabama', 'Alaska', 'Louisiana', 'Texas']:
        try:
            dry = dry.drop(f'{state}--Onshore', axis=1)
        except:
            print(f"for Onshore, state wasn't in df: {state}")
        try:
            dry = dry.drop(f'{state}--State Offshore', axis=1)
        except:
            print(f"for Offshore, state wasn't in df: {state}")

    # drop federal offshore attributed to states
    # (was only used 1992-1998)
    for col in [
        'Federal Offshore--Alabama',
        'Federal Offshore--Louisiana', 
        'Federal Offshore--Texas',
        ]:
        try:
            dry = dry.drop(col, axis=1)
        except:
            pass
        
    dry = dry.drop(['U.S.', 'Other States'], axis=1)
    dry = dry.rename(columns={'Federal Offshore--Gulf of Mexico': 'Fed GOM'})
    
    # select only the year of analysis
    dry_year = dry.copy().loc[data_year]

    return dry_year

In [ ]:
def read_international_dry_gas_production_and_consump(international_file, country):
    """
    In argument, units are Bcf. Converts to MMcf for output.
    """

    df = pd.read_csv(
        eia_path + international_file, 
        header=1)
    df = df.rename(columns={'Unnamed: 1': 'Category'})
    df = df.drop('API', axis=1)
    df['Category'] = df['Category'].str.strip()

    prod_row = df.loc[df['Category']=='Production'].index[0]
    dry_prod_row = prod_row + 2

    # check results
    if df.loc[dry_prod_row, 'Category'] == 'Dry natural gas (bcf)':
        dry_prod_year_bcf = df.at[dry_prod_row, str(data_year)]
    else:
        print("Error!" + f" Category result was unexpected; was {df.loc[dry_prod_row, 'Category']}")

    consump_row = df.loc[df['Category']=='Consumption'].index[0]
    dry_consump_row = consump_row + 1
    if df.loc[dry_consump_row, 'Category'] == 'Dry natural gas (bcf)':
        consump_year_bcf = df.at[dry_consump_row, str(data_year)]
    else:
        print("Error!" + f" Category result was unexpected; was {df.loc[dry_prod_row, 'Category']}")

    mmcf_per_bcf = 1000
    dry_prod_year_mmcf = dry_prod_year_bcf * mmcf_per_bcf
    consump_year_mmcf = consump_year_bcf * mmcf_per_bcf
    
    return(dry_prod_year_mmcf, consump_year_mmcf)

In [ ]:
def add_canada_mexico_dry_production_and_consump(
    dry_year, canada_dry_prod_year_mmcf, mexico_dry_prod_year_mmcf):

    # put production data into df dry_year
    dry_year.at['Canada'] = canada_dry_prod_year_mmcf
    dry_year.at['Mexico'] = mexico_dry_prod_year_mmcf
    
    return dry_year

In [ ]:
# run functions
dry = read_dry_gas_production_data(dry_file)
dry_year = clean_dry_prod_data_select_data_year(dry)

(canada_dry_prod_year_mmcf, canada_consump_year_mmcf) = read_international_dry_gas_production_and_consump(
    canada_international_file, 'Canada')
(mexico_dry_prod_year_mmcf, mexico_consump_year_mmcf) = read_international_dry_gas_production_and_consump(
    mexico_international_file, 'Mexico')
dry_year = add_canada_mexico_dry_production_and_consump(
    dry_year, canada_dry_prod_year_mmcf, mexico_dry_prod_year_mmcf)

## gross supply (dry production + imports)

In [ ]:
def calc_gross_supply_by_state(imports_year, dry_year):
    """
    Calculate gross supply for each state (and DC, Canada, and Mexico). 
    Gross supply = dry production + gross imports by each state (from other states or jurisdictions)

    Use dry production because that's "consumer-grade" gas,
    which goes into transmission pipelines and is sent across state lines;   
    EIA pipeline flows between states should most closely match dry gas.
    """
    
    # set threshold for import data to ignore
    # applying the threshold removes NaNs & tiny values that are neglibile and possibly erroneous
    supply_threshold = 500 # units MMcf

    gross_supply_list = []

    # iterate through all states to calculate gross supply for each
    for juris in all_jurisdictions_list:
    #     print(f"processing {juris}") # for db

        # create series for imports for each juris
        try:
            imports_juris = imports_year.loc[juris].T
        except:
            imports_juris = pd.Series()

        # filter to keep only positive values above supply_threshold
        imports_juris = imports_juris.loc[imports_juris > supply_threshold]

        for category in ['Interstate', 'International', 'International & Interstate']:
            try:
                imports_juris = imports_juris.drop(category)
            except:
                pass

        gross_supply_juris = imports_juris.copy()
        try:
            dry_juris = dry_year.at[juris]
        except:
            dry_juris = 0
        gross_supply_juris.at[juris] = dry_juris

        gross_supply_juris.name = juris
        gross_supply_juris = pd.DataFrame(gross_supply_juris).T
        gross_supply_juris.index.name = 'importing juris'
        gross_supply_juris.columns.name = 'from juris'

        gross_supply_list += [gross_supply_juris]

    gross_supply = pd.concat(gross_supply_list, sort=False)
    gross_supply.columns.name = 'from juris'
    
    # drop subtotals:
    for col in [
        'International & Interstate Receipts', 
        'International Receipts',
        'Interstate Receipts', 
        'All Countries', 
        'All States'
    ]:
        try:
            gross_supply = gross_supply.drop(col, axis=1)
        except:
            # these were already removed earlier; don't need to print line below
            # print(f"gross_supply didn't have col {col}")
            pass
    
    return gross_supply

In [ ]:
def attribute_supply_to_production_using_recursive_calculation(gross_supply, all_jurisdictions_list):
    """
    Create df for first-level supplies for a juris, as fractions.
    Each supply is labeled as import or production.
    For imports from each juris, trace back to origins of that juris's gross supply.
    Keep running while there are any imports listed in the df.
    
    Exports results to csv files for each state, since the results are very long.
    These files are then recompiled to calculate results.
    """

    supp_fract_prod_sums_list = [] # initialize
    
    # show threshold fraction for attributing consumption back to sources (e.g., 99%)
    # threshold_fract set in parameters at start of notebook
    print(f"threshold is {round(threshold_fract*100, 2)}%")

    # set timestamp once, for all states to be processed, so all saved files will have the same timestamp
    supp_fract_prod_timestamp = time.strftime('%Y-%m-%d_%H%M', time.localtime())

    for consump_juris in all_jurisdictions_list:
        # from gross supply df, calculate fractions of supply 
        # from each neighboring state (or own state production)
        supp_fract = create_supp_fract(gross_supply, consump_juris)
        
        # create df for state's own production
        supp_fract_prod_own = create_df_of_consuming_juris_own_production(supp_fract, consump_juris)

        # create df for recording production from all states
        # (starts with consuming state's own production)
        supp_fract_prod = supp_fract_prod_own.copy()

        # from supply fractions, create df of only imports;
        # this is an initial df that is added to iteratively below
        supp_fract_imp_init = supp_fract.copy().loc[supp_fract['type']=='import']
        supp_fract_imp_init['path'] = f'{consump_juris}-' + supp_fract_imp_init.index
        supp_fract_imp_init = supp_fract_imp_init.reset_index(drop=True)
        
        imports_init_shares = create_imports_init_shares(supp_fract_imp_init)

        # create list of jurisdictions to iterate through
        supp_fract_imp_init_juris = supp_fract_imp_init['path'].str.split('-').str[-1].tolist()
        
        for init_import_juris in supp_fract_imp_init_juris:
            # iterate through supplies to track origins of gas
            (supp_fract_prod_1juris, iter_num) = create_supp_fract_prod_1juris(
                init_import_juris,
                supp_fract_imp_init,
                imports_init_shares,
                consump_juris
            )
            
            # run test to check that scaled values total 100%
            test_scaled_values_sum_to_100_percent(supp_fract_prod_1juris, imports_init_shares, init_import_juris)

            # record number of iterations reached
            supp_fract_prod_1juris = record_number_of_iterations_reached(iter_num, supp_fract_prod_1juris)

            # put the final df supp_fract_prod_1juris into supp_fract_prod
            supp_fract_prod = supp_fract_prod.append(supp_fract_prod_1juris, sort=False)
            supp_fract_prod = supp_fract_prod.drop('share raw', axis=1)

        # run test
        test_consuming_state_total_supply_sums_to_100_percent(supp_fract_prod)

        # export supp_fract_prod
        supp_fract_prod.to_csv(
            inputs_path + 
            f'US gas model - {consump_juris} results supp_fract_prod {max_iter} iterations ({supp_fract_prod_timestamp}).csv', 
            index=False)

        # collect all dfs into a list of dfs
        supp_fract_prod_sums_list = collect_contributions_from_each_producing_state(
            supp_fract_prod, consump_juris, supp_fract_prod_sums_list)

    supp_fract_prod_sums_all = sum_all_state_dfs(supp_fract_prod_sums_list)
    
    # export
    supp_fract_prod_sums_all.to_excel(
        f'GIM gas flows supp_fract_prod_sums_all from gross trade for {data_year}, {max_iter} iterations ({supp_fract_prod_timestamp}).xlsx')

    return(supp_fract_prod_sums_all, supp_fract_prod_timestamp)
# end of attribute_supply_to_production_using_recursive_calculation

In [ ]:
def create_supp_fract(gross_supply, consump_juris):
    """
    Sub-function of attribute_supply_to_production_using_recursive_calculation
    """
    supp_fract_ser = gross_supply.loc[consump_juris].dropna() / gross_supply.loc[consump_juris].sum()
    # set name to be 'share raw'; overwrites consump_juris (e.g., 'California')
    supp_fract_ser.name = 'share raw'

    supp_fract = pd.DataFrame(supp_fract_ser)
    for row in supp_fract.index:
        if row == consump_juris:
            supp_fract.at[row, 'type'] = 'production'
        else:
            supp_fract.at[row, 'type'] = 'import'

    return supp_fract

In [ ]:
def create_df_of_consuming_juris_own_production(supp_fract, consump_juris):
    # start df for production with the consuming juris's own production
    supp_fract_prod_own = supp_fract.copy().loc[supp_fract['type']=='production']
    supp_fract_prod_own = supp_fract_prod_own.reset_index(drop=True)
    # for state's own production, no scaling required
    supp_fract_prod_own['share scaled'] = supp_fract_prod_own['share raw']
    supp_fract_prod_own['path'] = consump_juris
    supp_fract_prod_own['number of iterations'] = int(0)

    return supp_fract_prod_own

In [ ]:
def create_imports_init_shares(supp_fract_imp_init):
    # create df imports_init_shares for saving values for import shares from each immediate importer
    df = supp_fract_imp_init.copy()
    df['path end'] = df['path'].str.split('-').str[-1]
    df = df.set_index('path end')
    imports_init_shares = df['share raw']
    
    return imports_init_shares

In [ ]:
def record_number_of_iterations_reached(iter_num, supp_fract_prod_1juris):
    """
    Sub-function of attribute_supply_to_production_using_recursive_calculation.
    
    Puts number of iterations reached into dataframe.
    """
    max_iter_num = iter_num + 1
    try:
        max_iter_num = int(max_iter_num)
    except:
        print(f"couldn't convert max_iter_num to int: {max_iter_num}")
        pass
    supp_fract_prod_1juris['number of iterations'] = max_iter_num

    return supp_fract_prod_1juris

In [ ]:
def test_scaled_values_sum_to_100_percent(supp_fract_prod_1juris, imports_init_shares, init_import_juris):
    """
    Sub-function of attribute_supply_to_production_using_recursive_calculation
    """
    # TEST: add test to check that scaled values add up to 100%
    scaled_sum = supp_fract_prod_1juris['share scaled'].sum()
    import_share = imports_init_shares.loc[init_import_juris]
    if abs(scaled_sum - import_share) < 1e-6:
        pass
    else:
        print("Error!" + f" abs(1 - scaled_sum/import_share): {abs(1 - scaled_sum/import_share)}")
    # END OF TEST
    # no return

In [ ]:
def test_consuming_state_total_supply_sums_to_100_percent(supp_fract_prod):
    """
    Sub-function of attribute_supply_to_production_using_recursive_calculation
    """
    # TEST: check that for each consuming state, the supply shares add up to 1 (100%)
    share_sum = supp_fract_prod['share scaled'].sum()
    if abs(1 - share_sum) < 1e-6:
        pass
    else:
        print("Error!" + f" The sum of shares wasn't 100% (within rounding error); it was: {share_sum}")
    # END OF TEST
    # no return

In [ ]:
def create_supp_fract_prod_1juris(
    init_import_juris,
    supp_fract_imp_init,
    imports_init_shares,
    consump_juris
    ):  
    """
    Sub-function of attribute_supply_to_production_using_recursive_calculation
    """
    
    # select imports for the specified initial import juris
    supp_fract_imp_1juris = supp_fract_imp_init.loc[
        supp_fract_imp_init['path'].str.split('-').str[-1]==init_import_juris
    ].reset_index(drop=True)

    # initialize df for production data
    supp_fract_prod_1juris = pd.DataFrame()

    # iteratively calculate sources of supply for initial import juris
    for iter_num in range(0, max_iter):
        if len(supp_fract_prod_1juris)==0:
            supp_fract_prod_1juris_sum = 0 # initialize
        elif len(supp_fract_prod_1juris)>0:
            supp_fract_prod_1juris_sum = supp_fract_prod_1juris['share raw'].sum()
        else:
            print('Error!' + f" Unexpected value for len(supp_fract_prod_1juris): {len(supp_fract_prod_1juris)}")

        if supp_fract_prod_1juris_sum / imports_init_shares.loc[init_import_juris] < threshold_fract:
            if len(supp_fract_imp_1juris) > 0:
                # get data for first row of imports from df
                sel_row = 0
                source_juris = supp_fract_imp_1juris.loc[sel_row, 'path'].split('-')[-1]
                source_juris_share = supp_fract_imp_1juris.loc[sel_row, 'share raw']
                source_juris_path = supp_fract_imp_1juris.loc[sel_row, 'path'] # modified later

                # remove source_juris data from supp_fract_imp_1juris
                supp_fract_imp_1juris = supp_fract_imp_1juris.loc[supp_fract_imp_1juris.index!=sel_row]

                # run sub-function
                (supp_fract_prod_1juris, supp_fract_imp_1juris) = create_prod_and_imports_for_each_juris(
                    source_juris, 
                    source_juris_share, 
                    source_juris_path, 
                    supp_fract_prod_1juris, 
                    supp_fract_imp_1juris, 
                    gross_supply, 
                    all_jurisdictions_list)

            else:
                # len(supp_fract_imp_1juris)==0
                # no imports to process
                pass

        else:
            # supp_fract_prod['share raw'].sum() >= threshold_fract
            # assigned imports have reached the threshold; stop iterations                
            break
            
    # end of: for iter_num in range(0, max_iter)
    
    ratio_covered = supp_fract_prod_1juris['share raw'].sum() / imports_init_shares.loc[init_import_juris]       
    supp_fract_prod_1juris['share scaled'] = supp_fract_prod_1juris['share raw'] / ratio_covered
    print(f"for {consump_juris} via {init_import_juris}, completed {iter_num+1} iterations, covered {(ratio_covered*100).round(2)}%")

    return(supp_fract_prod_1juris, iter_num)

In [ ]:
def create_prod_and_imports_for_each_juris(
    source_juris, 
    source_juris_share, 
    source_juris_path, 
    supp_fract_prod_1juris, 
    supp_fract_imp_1juris, 
    gross_supply, 
    all_jurisdictions_list):
    """
    Sub-function of attribute_supply_to_production_using_recursive_calculation
    """
    if source_juris in all_jurisdictions_list:
        # get data from gross_supply for source state
        source_juris_sources = gross_supply.loc[source_juris].dropna()
        source_juris_total = gross_supply.loc[source_juris].sum()
        sources_fract = source_juris_sources / source_juris_total

        # multiply by share of imports (for given source juris)
        sources_scaled = sources_fract * source_juris_share

        # assign production or import label
        sources_scaled = pd.DataFrame(sources_scaled)
        for row in sources_scaled.index:
            if row == source_juris:
                sources_scaled.loc[row, 'type'] = 'production'
            else:
                sources_scaled.loc[row, 'type'] = 'import'

        # rename column from source_juris to 'share raw'
        # (for appending to dfs below)
        sources_scaled = sources_scaled.rename(columns={source_juris: 'share raw'})

        sources_scaled['path'] = f'{source_juris_path}-' + sources_scaled.index

        # if production, put value into supp_fract_prod_1juris
        sources_scaled_prod = sources_scaled.loc[sources_scaled['type']=='production']
        sources_scaled_prod = sources_scaled_prod.reset_index(drop=True)                        
        supp_fract_prod_1juris = supp_fract_prod_1juris.append(sources_scaled_prod, sort=False)

        # if import, put values into supp_fract_imp_1juris
        sources_scaled_imp = sources_scaled.loc[sources_scaled['type']=='import']
        supp_fract_imp_1juris = supp_fract_imp_1juris.append(sources_scaled_imp, sort=False)

    else:
        # source_juris is not in all_jurisdictions_list; 
        # source_juris is Fed GOM or foreign country (other than Canada or Mexico)
        supp_fract_prod_1juris = supp_fract_prod_1juris.reset_index(drop=True)
        next_index = supp_fract_prod_1juris.index.max() + 1

        supp_fract_prod_1juris.loc[next_index, 'share raw'] = source_juris_share
        supp_fract_prod_1juris.loc[next_index, 'type'] = 'production'
        supp_fract_prod_1juris.loc[next_index, 'path'] = source_juris_path + f'-{source_juris}'

    # reset index so there is always a 0th index for the next iteration
    supp_fract_imp_1juris = supp_fract_imp_1juris.reset_index(drop=True)

    return(supp_fract_prod_1juris, supp_fract_imp_1juris)

In [ ]:
def collect_contributions_from_each_producing_state(
    supp_fract_prod, consump_juris, supp_fract_prod_sums_list):
    """
    Sub-function of attribute_supply_to_production_using_recursive_calculation
    """
    # sum contributions from each producing state
    df = supp_fract_prod.copy()
    df['path_end'] = df['path'].str.split('-').str[-1]
    # for sums, change column name 'share scaled' to consump_juris
    df = df.rename(columns={'share scaled': consump_juris})
    df = df.groupby('path_end')[consump_juris].sum().sort_values(ascending=False)
    df = pd.DataFrame(df, columns=[consump_juris]).T
    df.index.name = 'consump juris'
    df.columns.name = 'prod juris'    
    supp_fract_prod_sums_list += [df]

    return supp_fract_prod_sums_list

In [ ]:
def sum_all_state_dfs(supp_fract_prod_sums_list):
    """
    Sub-function of attribute_supply_to_production_using_recursive_calculation
    """
    # after for loop for all states, sum all dfs in list
    supp_fract_prod_sums_all = pd.concat(supp_fract_prod_sums_list, sort=False)
    supp_fract_prod_sums_all.columns.name = 'producing juris'
    supp_fract_prod_sums_all.index.name = 'consuming juris'
    supp_fract_prod_sums_all = supp_fract_prod_sums_all.sort_index(axis=0)
    supp_fract_prod_sums_all = supp_fract_prod_sums_all.sort_index(axis=1)
    
    return supp_fract_prod_sums_all

In [ ]:
gross_supply = calc_gross_supply_by_state(imports_year, dry_year)
(supp_fract_prod_sums_all, 
 supp_fract_prod_timestamp) = attribute_supply_to_production_using_recursive_calculation(
    gross_supply, all_jurisdictions_list)

## TEST: check implied production by state
* Given the shares of each state's consumption that are supposed to come from each state's production...
* Can calculate what total production level is implied
* Multiply each state's consumption shares by its total consumption
* Then sum production from each producing state
* EIA data on consumption doesn't exactly match any of the production categories (e.g., dry), so things won't exactly match up
* But if within ~5%, probably good enough

In [ ]:
# # can import results from prior run
# supp_fract_prod_sums_all = pd.read_excel(
#     '/Users/masoninman/Dropbox/GEM/LCA of natural gas use/' + 
#     'US gas trade model supp_fract_prod_sums_all from gross trade 5000 iterations (2020-08-08_1039).xlsx'
# )
# supp_fract_prod_sums_all = supp_fract_prod_sums_all.set_index('consuming juris')

In [ ]:
def create_supp_dry_comparison(supp_fract_prod_sums_all, dry_year):
    
    # run sub-functions
    consump_juris_tot_all = read_eia_consump_data(canada_consump_year_mmcf, mexico_consump_year_mmcf)
    supp_quant_year_sums = calc_supp_quant_sums(supp_fract_prod_sums_all, consump_juris_tot_all)
    
    supp_dry_comparison = pd.concat([supp_quant_year_sums, dry_year], axis=1, sort=False)
    supp_dry_comparison = supp_dry_comparison.rename(columns={0: 'from consump', data_year: 'dry production'})

    # replace very small values with NaN
    for state in supp_dry_comparison.index:
        if supp_dry_comparison.at[state, 'from consump'] < 1:
            supp_dry_comparison.at[state, 'from consump'] = np.NaN

    supp_dry_comparison['fract diff'] = (supp_dry_comparison['from consump']-supp_dry_comparison['dry production'])/supp_dry_comparison['dry production']
    supp_dry_comparison = supp_dry_comparison.sort_values(by=['dry production'], ascending=False)

    # Note: would expect attributed volumes to be somewhat higher than dry production
    # because consumer-grade gas includes some ethane, which is excluded from dry production data
    
    return supp_dry_comparison

In [ ]:
def read_eia_consump_data(canada_consump_year_mmcf, mexico_consump_year_mmcf):
    """
    Sub-function of create_supp_dry_comparison
    
    Read data on consumption by state.
    
    From EIA page: Natural Gas Consumption by End Use
    https://www.eia.gov/dnav/ng/ng_cons_sum_dcu_STX_a.htm    
    """
    # import EIA data on gas consumption by state
    consump_juris_tot_all = pd.DataFrame() # initialize

    for state in states_list:
        state_abbrev = states_dict[state]
        consump_juris_df = pd.read_excel(
            gas_consump_data_path +
            f'NG_CONS_SUM_DCU_S{state_abbrev}_A.xls', 
            sheet_name='Data 1', 
            header=2,
        )

        consump_juris_df['year'] = consump_juris_df['Date'].astype(str).str.split('-').str[0].astype(int)

        consump_juris_df = consump_juris_df.set_index('year')
        consump_juris_df = consump_juris_df.drop('Date', axis=1)

        # keep only data_year
        consump_juris_df = consump_juris_df.loc[consump_juris_df.index==data_year]

        # pull out total consumption series
        consump_juris_df_tot = consump_juris_df[[f'{state} Natural Gas Total Consumption (MMcf)']]
        consump_juris_df_tot = consump_juris_df_tot.rename(columns={f'{state} Natural Gas Total Consumption (MMcf)': state})

        consump_juris_tot_all = pd.concat([consump_juris_tot_all, consump_juris_df_tot], axis=1)

    # EIA international data on consumption from
    if data_year > 2019:
        print("Error!" + f" Not set up to add consumption data for Canada and Mexico for data_year {data_year}.")
    else:
        consump_juris_tot_all.at[data_year, 'Canada'] = canada_consump_year_mmcf
        consump_juris_tot_all.at[data_year, 'Mexico'] = mexico_consump_year_mmcf

    # for rest of world, treat as though only consumption is from US exports
    # here we only need to attribute US exports back to producing US states, 
    consump_juris_tot_all.at[data_year, 'overseas'] = imports_overseas_year['overseas'].sum()
    
    return consump_juris_tot_all

In [ ]:
def calc_supp_quant_sums(supp_fract_prod_sums_all, consump_juris_tot_all):
    """Sub-function of create_supp_dry_comparison"""
    supp_quant_year = pd.DataFrame()

    for consump_juris in supp_fract_prod_sums_all.index:
        consump_juris_year_quant = consump_juris_tot_all.loc[data_year, consump_juris]
        supp_quant_state_year = consump_juris_year_quant * supp_fract_prod_sums_all.loc[supp_fract_prod_sums_all.index==consump_juris]

        supp_quant_year = supp_quant_year.append(supp_quant_state_year)

    supp_quant_year_sums = supp_quant_year.sum(axis=0)
    
    return supp_quant_year_sums

In [ ]:
supp_dry_comparison = create_supp_dry_comparison(supp_fract_prod_sums_all, dry_year)

# show:
supp_dry_comparison

In [ ]:
cont_us_only = supp_dry_comparison.drop(['Canada', 'Alaska'])[['from consump', 'dry production']].sum(axis=0)
print(f"pct diff: {round((cont_us_only.at['from consump'] - cont_us_only.at['dry production'])/cont_us_only.at['dry production'], 5)*100}%")

In [ ]:
# export
supp_dry_comparison.to_csv(f'GIM comparison of dry production vs modeled production by state data year {data_year} for {max_iter} iterations ({supp_fract_prod_timestamp}).csv')